In [ ]:
!pip install -r sample_data/requirements.txt

In [ ]:
##Text loader
from langchain_community.document_loaders import TextLoader
txt_loader = TextLoader('sample_data/speech.txt')
txt_documents = txt_loader.load()
txt_documents

In [ ]:
##PDF loader
from langchain_community.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader("sample_data/attention.pdf")
pdf_documents= pdf_loader.load()
pdf_documents

In [ ]:
##Web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
web_loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                           bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                                   class_=("post-title","post-content","post-header")
                               ))
                           )
web_documents = web_loader.load()
web_documents

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [21]:
##Arxiv Loader
from langchain_community.document_loaders import ArxivLoader

# Supports all arguments of `ArxivAPIWrapper`
arxiv_loader = ArxivLoader(
    query="reasoning",
    load_max_docs=2,
    # doc_content_chars_max=1000,
    # load_all_available_meta=False,
    # ...
)
arxiv_documents = arxiv_loader.load()
arxiv_documents[0]

Document(metadata={'Published': '2024-10-16', 'Title': 'Graph-constrained Reasoning: Faithful Reasoning on Knowledge Graphs with Large Language Models', 'Authors': 'Linhao Luo, Zicheng Zhao, Chen Gong, Gholamreza Haffari, Shirui Pan', 'Summary': 'Large language models (LLMs) have demonstrated impressive reasoning\nabilities, but they still struggle with faithful reasoning due to knowledge\ngaps and hallucinations. To address these issues, knowledge graphs (KGs) have\nbeen utilized to enhance LLM reasoning through their structured knowledge.\nHowever, existing KG-enhanced methods, either retrieval-based or agent-based,\nencounter difficulties in accurately retrieving knowledge and efficiently\ntraversing KGs at scale. In this work, we introduce graph-constrained reasoning\n(GCR), a novel framework that bridges structured knowledge in KGs with\nunstructured reasoning in LLMs. To eliminate hallucinations, GCR ensures\nfaithful KG-grounded reasoning by integrating KG structure into the LLM

In [23]:
print(arxiv_documents[0].metadata)

{'Published': '2024-10-16', 'Title': 'Graph-constrained Reasoning: Faithful Reasoning on Knowledge Graphs with Large Language Models', 'Authors': 'Linhao Luo, Zicheng Zhao, Chen Gong, Gholamreza Haffari, Shirui Pan', 'Summary': 'Large language models (LLMs) have demonstrated impressive reasoning\nabilities, but they still struggle with faithful reasoning due to knowledge\ngaps and hallucinations. To address these issues, knowledge graphs (KGs) have\nbeen utilized to enhance LLM reasoning through their structured knowledge.\nHowever, existing KG-enhanced methods, either retrieval-based or agent-based,\nencounter difficulties in accurately retrieving knowledge and efficiently\ntraversing KGs at scale. In this work, we introduce graph-constrained reasoning\n(GCR), a novel framework that bridges structured knowledge in KGs with\nunstructured reasoning in LLMs. To eliminate hallucinations, GCR ensures\nfaithful KG-grounded reasoning by integrating KG structure into the LLM\ndecoding process

In [26]:
##Wikipedia loader
from langchain_community.document_loaders import WikipediaLoader
docs = WikipediaLoader(query="Generative AI", load_max_docs=2).load()
len(docs)
print(docs)

[Document(metadata={'title': 'Generative artificial intelligence', 'summary': 'Generative artificial intelligence (generative AI, GenAI, or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.  \nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora. Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.\nGenerat

In [27]:
print(docs[0].metadata)

{'title': 'Generative artificial intelligence', 'summary': 'Generative artificial intelligence (generative AI, GenAI, or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.  \nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora. Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.\nGenerative AI has uses acr